<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/2021notebooks/2021cnps_demo_animal_word_association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021cnps 配布用 動物概念の単語連想課題のデモ <font color="teal">あつまれどうぶつの森モデル</font>

- author: 浅川伸一
- date: 2021_0830
- License: MIT license
- filename: 2021cnps_demo_animal_word_association.ipynb

どうぶつの名前を想起する課題のモデル化の試み。

以下モデルの概要

1. 多くの動物単語ベクトルで張られる空間へ単語ベクトルを射影し，
2. その射影空間内で類似したベクトルの想起 model
3. 各ベクトルの類似度をソフトマックス関数を用いて確率密度に変換
4. 変換した確率密度を元にサンプリングする


## 意味空間での操作，射影行列による語想起モデル

1. WAB の `動物の名前を一分間でできる限り想起してください` 課題は，この部分空間内での単語検索であると考えた。
1. 射影行列を用いて，意味空間を分割します。動物の名前で構成される空間をそれ以外の空間に分割する。
1. ここに示す語想起モデルでは，単語間の距離をソフトマックス関数によって多項分布の確率密度に変換し，
1. 得られた多項分布の確率密度関数からサンプリングとして単語想起を捉えた。
1. このとき，たとえば `ネコ` を手がかりとして与えた場合，`ネコ` に近い単語として `マタタビ` が想起されては困ると考えた。
1. 動物の名前で構成された部分空間を考えれば，`マタタビ` は想起されにくくなるだろうと考えられる。

1. おそらく，側頭葉に蓄えられている意味の表象に対して，前頭葉からのフィードバックが，この探索空間を限定することに相当し，
1. シリトリ，たとえば `カ` で始まる単語に限定する単語想起なども同様の操作で実現可能だろうと考えられる。
1. 基本的で単純なアイデアだが，意味空間とその上位に位置するメタ知識，あるいはモニターとか文脈とかと言い換えても良い，という脳内機序をモデル化できるだろうと考えている。

## 直交射影空間

線形代数の教科書を紐解けば，任意のベクトルの組 $X$ の成す空間への直交射影行列は $X(X^\top X)^{-1}X^\top$ で表される。
この行列を $P$ とすれば冪等性 $P^{n}=P$ と，その直交補空間 $C=I-P$ の冪等性 $C^n=C$ が成り立つ。
ここで $n$ は任意の自然数である。
これらの射影行列を用いて任意の実ベクトル $y$ は $\hat{y}=Py$, $y^c=Cy=(I-P)y$ とすると $\hat{y}\perp y^{c}$ すなわち $ \hat{y}^\top y^{c} = 0$ が成り立つ。
ここで $I$ は単位行列を表すものとする。

ここで $X$ が一つのベクトルである特殊な場合を考えれば，射影行列は次式のようになる:
$$
x(x^\top x)x^\top =\frac{xx^\top}{x^\top x},
$$
上式，分母は自らとの内積すなわちベクトルの長さに対応する量である。
$x$ が実ベクトルであれば，自身との内積は 非負 ($x^\top x\ge0$) となる。
すなわち，このときの射影行列は，自分自身で構成される空間への射影ベクトルを自分自身の長さだけ大きくなるので，
その分を割って調整していることとなる。


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import os
import sys
import time
import gensim

In [ ]:
# word2vec の訓練済モデルを入手
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid300_win20_neg20_sgns.bin.gz'
#!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.g

--2021-08-28 23:37:22--  http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
Resolving www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)... 160.13.90.21
Connecting to www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)|160.13.90.21|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz [following]
--2021-08-28 23:37:23--  https://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz
Connecting to www.cis.twcu.ac.jp (www.cis.twcu.ac.jp)|160.13.90.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1369049082 (1.3G) [application/x-gzip]
Saving to: ‘2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz’

2017Jul_jawiki-waka 100%[===================>]   1.27G  17.6MB/s    in 77s     

2021-08-28 23:38:40 (17.0 MB/s) - ‘2017Jul_jawiki-wakati_neologd_hid200_win20

In [ ]:
word2vec_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file, 
                                                        encoding='utf-8', 
                                                        unicode_errors='replace', 
                                                        binary=True)
w2v = model

In [ ]:
class animal_forest():
    """
    動物の自由連想課題のシミュレーション
    """
    def __init__(self, w2v=w2v, animals=False):
        
        self.w2v = w2v
        if not animals:
            self.animals = ['おたまじゃくし', 'アオウミガメ', 'アオサギ', 'アオダイショウ', 'アカアシシギ', 
                            'アカエイ', 'アカオネッタイチョウ', 'アカガエル', 'アカギツネ', 'アカゲザル', 
                            'アカゲラ', 'アカシカ', 'アカテガニ', 'アカネズミ', 'アカボウクジラ', 'アカントステガ', 
                            'アクロカントサウルス', 'アザラシ', 'アシカ', 'アジサシ', 'アナウサギ', 
                            'アナグマ', 'アノマロカリス', 'アフリカゾウ', 'アフリカツメガエル', 'アブラツノザメ', 
                            'アホウドリ', 'アマガエル', 'アマツバメ', 'アマモ', 'アミメニシキヘビ', 'アメフラシ', 
                            'アメリカバイソン', 'アメリカワシミミズク', 'アメンボ', 'アメーバ', 'アユ', 
                            'アライグマ', 'アリクイ', 'アリゲーターガー', 'アリジゴク', 'アルマジロ', 'アロサウルス', 
                            'アロワナ', 'アンキロサウルス', 'アンコウ', 'アンテオサウルス', 'アンドリューサルクス', 
                            'アンモナイト', 'アーケオプテリクス', 'アーケロン', 'アースロプレウラ', 'イエネコ', 
                            'イクチオサウルス', 'イクチオステガ', 'イグアナ', 'イグアノドン', 'イサキ', 'イシガイ', 
                            'イシサンゴ', 'イソギンチャク', 'イタセンパラ', 'イタチザメ', 'イッカク', 'イヌ', 
                            'イヌワシ', 'イノシシ', 'イボニシ', 'イモムシ', 'イモリ', 'イリエワニ', 
                            'イリオモテヤマネコ', 'イルカ', 'イロワケイルカ', 'イワシ', 'インコ', 'インドガビアル', 
                            'インドクジャク', 'インドハッカ', 'ウイルス', 'ウグイ', 'ウサギ', 'ウシ', 
                            'ウシガエル', 'ウズラ', 'ウツボ', 'ウナギ', 'ウバザメ', 'ウマ', 'ウミウシ', 
                            'ウミガメ', 'ウミクワガタ', 'ウミサソリ', 'ウミタナゴ', 'ウミツバメ', 'ウミニナ', 
                            'ウミヘビ', 'ウミユリ', 'ウンピョウ', 'エイリアン', 'エウステノプテロン', 'エゾヒグマ', 
                            'エダフォサウルス', 'エチゼンクラゲ', 'エナガ', 'エピオルニス', 'エボシガイ', 'エミュー', 
                            'オウムガイ', 'オオウナギ', 'オオウミガラス', 'オオカミ', 'オオコウモリ', 'オオサンショウウオ', 
                            'オオセ', 'オオタカ', 'オオツノシカ', 'オオトカゲ', 'オオハシ', 'オオハシウミガラス', 'オオバン', 
                            'オオフラミンゴ', 'オオマリコケムシ', 'オオメジロザメ', 'オオヤマネコ', 'オキアミ', 'オジロジカ', 
                            'オジロワシ', 'オタマジャクシ', 'オットセイ', 'オナガガモ', 'オニアンコウ', 'オニオオハシ', 
                            'オニヒトデ', 'オポッサム', 'オマキザル', 'オランウータン', 'オルガネラ', 'オヴィラプトロサウルス', 
                            'カイウサギ', 'カイギュウ', 'カイメン', 'カエル', 'カギムシ', 'カサゴ', 'カジキ', 'カタクチイワシ', 
                            'カタツムリ', 'カダヤシ', 'カッコウ', 'カツオクジラ', 'カツオドリ', 'カツオノエボシ', 'カナダガン', 
                            'カニ', 'カニクイアザラシ', 'カニクイザル', 'カニダマシ', 'カバ', 'カピバラ', 'カブトガニ', 
                            'カブトムシ', 'カマキリ', 'カメ', 'カメムシ', 'カモ', 'カモノハシ', 'カヤネズミ', 'カライワシ', 
                            'カラシン', 'カラス', 'カルカロドントサウルス', 'カルガモ', 'カレイ', 'カワウソ', 'カワガラス', 
                            'カワゲラ', 'カワスズメ', 'カワセミ', 'カワニナ', 'カワハギ', 'カワラヒワ', 'カンガルー', 
                            'ガチョウ', 'キジ', 'キジバト', 'キセルガイ', 'キタキツネ', 'キタリス', 'キツツキ', 'キツネ', 
                            'キツネザル', 'キバタン', 'キュウリウオ', 'キンクロハジロ', 'キンチャクダイ', 'ギガノトサウルス', 
                            'ギフチョウ', 'ギンダラ', 'ギンブナ', 'クサガメ', 'クサフグ', 'クジラ', 'クニマス', 'クマ', 
                            'クマネズミ', 'クマノミ', 'クモ', 'クモザル', 'クラゲ', 'クラーケン', 'クリオネ', 'クリダグナサス', 
                            'クリーパー', 'クルマエビ', 'クロツヤムシ', 'クロトキ', 'クロナガアリ', 'クロマグロ', 'クワガタムシ', 
                            'グッピー', 'グリプトドン', 'グリーンイグアナ', 'グンカンドリ', 'ケツァルコアトルス', 'ケンミジンコ', 
                            'ケンムン', 'ケープハイラックス', 'ケープペンギン', 'ゲッチョ', 'ゲラダヒヒ', 'ゲンゴロウ', 
                            'コアジサシ', 'コイ', 'コウイカ', 'コウノトリ', 'コウモリ', 'コオロギ', 'コクチョウ', 
                            'コゲラ', 'コサギ', 'コジュケイ', 'コノドント', 'コビトカバ', 'コブハクチョウ', 'コモドオオトカゲ', 
                            'コンゴウインコ', 'コンプソグナトゥス', 'ゴイサギ', 'ゴカイ', 'ゴキブリ', 'ゴマフアザラシ', 
                            'ゴルゴサウルス', 'ゴルゴノプス', 'ゴンズイ', 'ゴンドウクジラ', 'サイチョウ', 'サカマキガイ', 
                            'サケ', 'サケイ', 'サソリ', 'サメ', 'サル', 'サワガニ', 'サンゴ', 'サンショウウオ', 'ザトウクジラ', 
                            'ザリガニ', 'シアノバクテリア', 'シカ', 'シカクナマコ', 'シギダチョウ', 'シクリッド', 'シジュウカラ', 
                            'シノカリオプテリクス', 'シノサウロプテリクス', 'シファクティヌス', 'シマウマ', 'シマフクロウ', 
                            'シャチ', 'シュモクザメ', 'ショウジョウバエ', 'シラウオ', 'シラミ', 'シロアリ', 'シロイルカ', 
                            'シロウオ', 'シロサイ', 'シロナガスクジラ', 'シーサーペント', 'シーラカンス', 'ジャガーネコ', 
                            'ジュゴン', 'ジリス', 'ジンベエザメ', 'スイギュウ', 'スカフォグナトゥス', 'スクミリンゴガイ', 
                            'スジイルカ', 'スジエビ', 'スズガモ', 'スズメ', 'スズメガ', 'スズメダイ', 'スティラコサウルス', 
                            'ステゴサウルス', 'ストロマトライト', 'スナギンチャク', 'スナネズミ', 'スナメリ', 'スネール', 
                            'スピノサウルス', 'スミロドン', 'スローロリス', 'セイウチ', 'セイヨウミツバチ', 'セグロカモメ', 
                            'セミクジラ', 'センザンコウ', 'ソコダラ', 'ゾウ', 'ゾウアザラシ', 'ゾウリムシ', 'タイコウチ', 
                            'タガメ', 'タキタロウ', 'タコ', 'タスマニアデビル', 'タツノオトシゴ', 'タナゴ', 'タニシ', 
                            'タヌキ', 'タンチョウ', 'ダイオウイカ', 'ダイサギ', 'ダチョウ', 'ダツ', 'ダマジカ', 'ダルマザメ', 
                            'ダンクルオステウス', 'ダンゴムシ', 'チューブワーム', 'チョウ', 'チョウザメ', 'チョウチョウウオ', 
                            'チョウチンアンコウ', 'チリモン', 'チンチラ', 'チンパンジー', 'ツェツェバエ', 'ツキノワグマ', 
                            'ツチガエル', 'ツチスドリ', 'ツチノコ', 'ツチブタ', 'ツノザメ', 'ティラノサウルス', 'ティロサウルス', 
                            'テッポウエビ', 'テナガエビ', 'テナガザル', 'テノントサウルス', 'テミンク', 'テンジクネズミ', 
                            'テントウムシ', 'ディアデクテス', 'ディキノドン', 'ディノケファルス', 'ディモルフォドン', 
                            'デイノテリウム', 'デイノニクス', 'デジモン', 'デスモスチルス', 'デルフチア・アシドボランス', 
                            'トカゲ', 'トガリネズミ', 'トゲウオ', 'トビケラ', 'トビハゼ', 'トリガイ', 'トリケラトプス', 
                            'トルヴォサウルス', 'トロオドン', 'ドクトカゲ', 'ドジョウ', 'ドブネズミ', 'ドロマエオサウルス', 
                            'ドードー', 'ナガスクジラ', 'ナキウサギ', 'ナマケモノ', 'ナマコ', 'ナマズ', 'ナメクジ', 
                            'ニゴイ', 'ニザダイ', 'ニシツノメドリ', 'ニシローランドゴリラ', 'ニジマス', 'ニタリクジラ', 
                            'ニホンアカガエル', 'ニホンイシガメ', 'ニホンウナギ', 'ニホンカナヘビ', 'ニホンカワウソ', 
                            'ニホンザリガニ', 'ニホンザル', 'ニホントカゲ', 'ニホンヤモリ', 'ニューギニアヤリガタリクウズムシ', 
                            'ニワトリ', 'ヌタウナギ', 'ヌマエビ', 'ネコ', 'ネズッポ', 'ネズミ', 'ネズミイルカ', 'ネズミザメ', 
                            'ネッシー', 'ノウサギ', 'ノコギリエイ', 'ノネズミ', 'ノーザンパイク', 'ハイイロガン', 'ハイギョ', 
                            'ハエ', 'ハオリムシ', 'ハキリアリ', 'ハクジラ', 'ハクビシン', 'ハゲワシ', 'ハコフグ', 'ハシビロガモ', 
                            'ハシビロコウ', 'ハジラミ', 'ハゼ', 'ハタネズミ', 'ハダカイワシ', 'ハダカデバネズミ', 'ハチ', 
                            'ハチドリ', 'ハツカネズミ', 'ハト目', 'ハナムグリ', 'ハリネズミ', 'ハリモグラ', 'ハーテビースト', 
                            'バイカルアザラシ', 'バクテリア', 'バッタ', 'バンドウイルカ', 'パキケトゥス', 'パキケファロサウルス', 
                            'パレイアサウルス', 'ヒキガエル', 'ヒクイドリ', 'ヒグマ', 'ヒゲクジラ', 'ヒツジ', 'ヒト', 'ヒトデ', 
                            'ヒドリガモ', 'ヒナモロコ', 'ヒヒ', 'ヒプシロフォドン', 'ヒヨケザル', 'ヒヨドリ', 'ヒラメ',
                            'ピコプランクトン', 'ピラニア', 'ピラルクー', 'ピーシーズ', 'フェイゾン', 'フエガラス', 
                            'フォルスラコス', 'フクロウ', 'フクロウオウム', 'フクロオオカミ', 'フクロネコ', 'フクロネズミ', 
                            'フクロモモンガ', 'フグ', 'フジツボ', 'フデイシ', 'フルマカモメ', 'ブタ', 'ブチハイエナ', 
                            'ブッポウソウ', 'ブラキオサウルス', 'プテラノドン', 'プテロダクティルス', 'プラナリア', 
                            'プランクトン', 'プレシオサウルス', 'プレソドン', 'プレーリードッグ', 'プロガノケリス', 
                            'プロトプラスト', 'ヘコアユ', 'ヘスペロルニス', 'ヘビ', 'ヘビクイワシ', 'ベニイロフラミンゴ', 
                            'ベニザケ', 'ベンガルトラ', 'ベントス', 'ペッカリー', 'ペット', 'ペラゴルニス', 'ペリカン', 
                            'ペリット', 'ペンギン', 'ホウボウ', 'ホオダレムクドリ', 'ホシハジロ', 'ホシムクドリ', 'ホタル', 
                            'ホタルイカ', 'ホッキョクギツネ', 'ホッキョクグマ', 'ホホジロザメ', 'ホラアナグマ', 'ホライモリ', 
                            'ホワイトバス', 'ホンソメワケベラ', 'ボノボ', 'ボラジョ', 'ポリプテルス', 'マイワシ', 'マガモ', 
                            'マジュンガサウルス', 'マダイ', 'マダコ', 'マダニ', 'マッコウクジラ', 'マナヅル', 'マナティー', 
                            'マハゼ', 'マプサウルス', 'マルハナバチ', 'マレーグマ', 'マンジュウダイ', 'マンモス', 
                            'マーモセット', 'マーモット', 'マーレーコッド', 'ミクロラプトル', 'ミサゴ', 'ミシシッピアカミミガメ', 
                            'ミジンコ', 'ミズガメ', 'ミズナギドリ', 'ミツバチ', 'ミトコンドリア', 'ミドリムシ', 
                            'ミナミゾウアザラシ', 'ミミズ', 'ミミズク', 'ミュールジカ', 'ミーアキャット', 'ムカシトカゲ', 
                            'ムカシトンボ', 'ムカデ', 'ムクドリ', 'ムササビ', 'ムツゴロウ', 'ムラサキウニ', 'メガテリウム', 
                            'メガロドン', 'メジロザメ', 'メダカ', 'メトリオリンクス', 'メトロイド', 'メバル', 'モエビ', 
                            'モクズガニ', 'モグラ', 'モササウルス', 'モツゴ', 'モノアラガイ', 'モモイロペリカン', 
                            'モルモット', 'モンガラカワハギ', 'モーグリ', 'ヤギ', 'ヤシガニ', 'ヤツメウナギ', 'ヤドカリ', 
                            'ヤノマミ', 'ヤブカ', 'ヤマアカガエル', 'ヤマアラシ', 'ヤマカガシ', 'ヤマシギ', 'ヤマトシジミ', 
                            'ヤモリ', 'ヤンバルクイナ', 'ユスリカ', 'ユリカモメ', 'ヨコエソ', 'ヨコエビ', 'ヨコバイ', 
                            'ヨシノボリ', 'ヨロイザメ', 'ヨーロッパヒメウ', 'ライチョウ', 'ラッコ', 'ランフォリンクス', 
                            'リオプレウロドン', 'リャマ', 'ルリオーストラリアムシクイ', 'ルリビタキ', 'レイクトラウト', 
                            'レイヨウ', 'レッサーパンダ', 'レプトケファルス', 'ロボク', 'ワシミミズク', 'ワタリガニ', 
                            'ワニ', 'ワニトカゲギス', 'ワモンアザラシ', 'ワライカワセミ', 'ワラジムシ', 'ヴェロキラプトル', 
                            '回虫', '始祖鳥', '山羊', '巻き貝', '巻貝', '栗鼠', '毛虫', '犬', '猫', '猿', '硬骨魚', 
                            '蛇', '蛙', '蛾', '蜘蛛', '蠕虫', '豚', '象', '貝', '野ネズミ', '金魚', '鯨']
            # animals の リストからどうぶつの射影行列 P を作成
            P = np.zeros((len(self.animals), self.w2v.vector_size), dtype=np.float)
            for i, animal in enumerate(self.animals):
                P[i] = np.copy(self.w2v[animal])
            self.Proj = np.dot(P.T,P) 

    def word_assoc(self, word, Proj=None, n_seq=5, topn=32, beta=0.03, seed=int(time.time()), verbose=False, w2v=w2v):
        """
        自由連想課題のシミュレーション: 単語 word を刺激語として逐次自由連想を繰り返す
        パラメータ:
            word: 刺激語 
            Proj: 動物名空間への射影行列
            n_seq: 自由連想を何回続けるか (int)
            topn: 想起する単語の最大数，上位 topn 個の単語までが考慮される(int)
            beta: 温度パラメータ，低いと決定論的に振る舞い，高いとその都度振る舞いが異なる(float)
            seed: 乱数の種 指定しなければ現在の時刻を用いる。指定すると再現性が保たれる (int) 
            verbose: 冗長な出力を行う (bool)
            w2v: word2vec の本体
        """
        if Proj is None:
            Proj = self.Proj

        np.random.seed(seed)  # 乱数のタネを設定
        ret = []  # 戻り値
        for i in range(n_seq):
            #vec = self.w2v[word]  # ターゲット語のベクトル
            vec = np.dot(Proj, self.w2v[word])             # 射影行列をかけて 動物空間へ射影する
            names = self.w2v.similar_by_vector(vec)
            entries = np.array([x[0] for x in names])[1:]  # 最初のエントリーだけ外す [1:] の部分，
                                                           # 保続モデルにするなら  [1:] を削除してください
            sims = np.array([x[1] for x in names])[1:]
            probs = np.exp(sims/beta)/np.exp(sims/beta).sum()  # softmax
            word  = np.random.choice(entries,p=probs)
            ret.append(word)
            if verbose:
                print(word, end="")
                print(' :', entries.index(word)+1, entries, probs)
        return ret
        

In [ ]:
simulation = animal_forest()
print(simulation.word_assoc('ネコ', n_seq=3, beta=0.05)) # 3 個だけ想起してくださいの場合

print(simulation.word_assoc('魚', n_seq=20))  # 20 個想起してもらう場合

['フクロウ', 'ウツボ', 'ヘビ']
['スジエビ', 'スジエビ', 'スジエビ', 'ザリガニ', 'ザリガニ', 'サンショウウオ', 'サンショウウオ', 'アリクイ', 'カワウソ', 'ウツボ', 'サンショウウオ', 'カワウソ', 'ヘビ', 'アリクイ', 'ワニ', 'アリクイ', 'ザリガニ', 'ザリガニ', 'カワウソ', 'ヘビ']


In [ ]:
key_word = input('種となる単語を乳力してください: ')
print(simulation.word_assoc(key_word, n_seq=10, beta=0.05)) 


種となる単語を乳力してください: サザエ
['ウミヘビ', 'アリクイ', 'ヘビ', 'ザリガニ', 'ザリガニ', 'サンショウウオ', 'カワウソ', 'ヘビ', 'ダチョウ', 'アリクイ']


In [ ]:
vegetable_names = [
    'あずき', 'いも', 'いんげん', 'いんげん豆', 'うずら豆', 'うど', 'えだまめ', 'えんどう豆', 'おいも', 'おから', 'かぶ', \
    'かぼちゃ',  'かんぴょう', 'きのこ', 'きゅうり', 'きんとき', 'ぎんなん', 'ごぼう', 'ごま', 'さつまいも', 'さつま芋', \
    'さといも', 'さとうきび', 'さやいんげん',  'しいたけ', 'しそ', 'しめじ', 'じゃがいも', 'ずんだ', 'そら豆', 'たけのこ', \
    'たまねぎ', 'だいこん', 'とうきび', 'とうもろこし', 'どんぐり', 'にんじん', 'にんにく', 'ねぎ', 'ねなし', 'のらぼう菜', \
    'はくさい', 'ほうれん草', 'まめ', 'みょうが', 'もやし', 'もろこし', 'やさい', 'れんこん', 'アカザ', 'アカザ科', 'アカツメクサ', \
    'アカヤシオ', 'アサイー', 'アサガオ', 'アサツキ', 'アシタバ', 'アジサイ', 'アスパラガス', 'アズキ', 'アフリカヒゲシバ', 'アブラナ', \
    'アブラヤシ', 'アボカド', 'アマチャヅル', 'アマランサス', 'アミガサタケ', 'アリタソウ', 'アロエ', 'アーモンド', 'イグサ', 'イタリアンパセリ', \
    'イナゴマメ', 'イヌガラシ', 'イヌツゲ', 'イノンド', 'イモ', 'インゲン', 'インゲンマメ', 'インゲン豆', 'インジェラ', 'ウイキョウ', \
    'ウチワサボテン', 'ウマゴヤシ', 'エゴマ', 'エシャロット', 'エゾカンゾウ', 'エノキタケ', 'エンドウ', 'エンドウマメ', 'オオウバユリ', 'オオバ', \
    'オオバナノエンレイソウ', 'オオムギ','オクラ', 'オタネニンジン', 'オナモミ', 'オニオンリング', 'オニグルミ', 'オリーブ',  'カオソーイ', \
    'カキノキ', 'カサスゲ', 'カシューナッツ', 'カブの葉', 'カボチャ', 'カラシナ', 'カラスノエンドウ', 'カラスムギ', 'カリフラワー', \
    'キクイモ', 'キノコ', 'キノコ類', 'キマメ', 'キムチ', 'キャセロール', 'キャッサバ', 'キャベツ', 'キュウリ', 'キョフテ', 'キンシウリ', \
    'キンセンカ', 'キンマ', 'キンレンカ', 'ギョウジャニンニク', 'ギリシャ料理', 'クルミ', 'クレソン', 'クワイ', 'グリンピース', 'ケシ', \
    'コケモモ', 'コマツナ', 'コムギ', 'ゴボウ', 'ゴマ', 'サツマイモ', 'サトイモ', 'サトウキビ', 'サヤインゲン', 'シイタケ', 'シシトウガラシ', \
    'シメジ', 'シュンギク', 'ショウガ', 'シロウリ', 'ジャガイモ', 'ジュンサイ', 'スイートコーン', 'スズメノエンドウ', 'スペインカンゾウ', \
    'スモモ', 'ズッキーニ', 'セイヨウカラシナ', 'セイヨウスモモ', 'セイヨウノコギリソウ', 'セイヨウミザクラ', 'セモリナ', 'セロリ', 'センナ', \
    'ソラマメ', 'タケノコ', 'タマネギ', 'タラの芽', 'タラノキ',  'タロイモ', 'ダイコン', 'チコリー', 'チンゲンサイ', 'テンサイ', 'デュラムコムギ', \
    'デーツ',  'トウゴマ', 'トウモロコシ', 'トマト', 'トリュフ', 'トルコギキョウ', 'ドングリ', 'ナガイモ', 'ナス', 'ナスタチウム', 'ナタマメ', \
    'ナッツ',  'ニガウリ', 'ニラ', 'ニンジン', 'ニンニク',  'ネギ', 'ハクサイ', 'ハスイモ', 'ハツカダイコン', 'ハトムギ', 'パセリ', 'パプリカ', \
    'ヒヨコマメ', 'ヒラタケ', 'ヒラマメ',  'ピーナッツ', 'ピーナツ', 'ピーマン', 'ヘーゼルナッツ', 'ホウレンソウ', 'ホオズキ', 'ホワイトアスパラ', \
    'ホースラディッシュ', 'ポテト',  'ポルチーニ', 'マカダミアナッツ',  'マクワウリ', 'マッシュルーム', 'マツタケ', 'ミズナ', 'ミニトマト', \
    'ミョウガ', 'ムギ', 'ムラサキウマゴヤシ', 'メキャベツ', 'モヤシ', 'モロコシ', 'モロヘイヤ', 'ヤマイモ', 'ヤマノイモ', 'ヤマブドウ', 'ヤマユリ', \
    'ヤムイモ', 'ヨモギ', 'ライマメ', 'ライムギ', 'ライ麦',  'ラッカセイ', 'ラッキョウ', 'ルッコラ', 'レタス', 'レモングラス', 'レンコン', \
    'レンズマメ', 'レンズ豆', 'ワケギ',  '二条大麦', '人参', '仙草', '入元', '刻みネギ', '南瓜', '大根', '大棗', '大蒜', '大豆', '大賀ハス', \
    '大麦', '守口大根', '小松菜', '小豆',  '小麦', '山椒', '山菜', '朝鮮人参', '木の実', '木の芽', '松の実', '松茸', '枝豆', '栗', '根菜', \
    '椎茸', '玄米', '玉ねぎ', '玉ネギ', '玉菜', '玉葱', '瓜', '生姜', '白ごま', '白インゲン', '白ゴマ', '白ネギ', '穀物', '竹の子', '粟', \
    '紫キャベツ', '紫蘇','緑豆', '緑黄色野菜', '練馬大根', '聖護院大根',  '胡瓜', '胡麻',  '自然薯', '舞茸', '芋', '花野菜', '芽キャベツ',  \
    '茄子', '菜種',  '落花生', '葉っぱ', '葉ネギ', '葉菜類',  '葱','蒟蒻', '蓮根', '蔘', '蔬菜', '薬用人参', '赤カブ', '赤ピーマン', \
    '里芋', '野沢菜', '野草', '野菊', '野菜', '金時豆', '銀杏', '長ネギ', '長芋','青ねぎ', '青ネギ', '青菜','食用キノコ', '食用菊', \
    '香味野菜', '香草', '馬鈴薯', '高原野菜', '高菜',  '高麗人参', '麦', '黒ごま', '黒コショウ', '黒ゴマ', '黒大豆', '黒砂糖', '黒米', 
    '黒胡椒', '黒蜜', '黒豆']

P = np.zeros((len(vegetable_names), w2v.vector_size), dtype=np.float)
for i, veg in enumerate(vegetable_names):
    P[i] = np.copy(w2v[veg])
vegP = np.dot(P.T,P) 

veg_assoc = animal_forest()
veg_assoc.word_assoc(word='白菜', Proj=vegP)


['ハクサイ', 'ソラマメ', 'ソラマメ', 'キャベツ', 'キャベツ']

In [ ]:
key_word = input('種となる単語を入力してください: ')
print(veg_assoc.word_assoc(key_word, Proj=vegP, n_seq=10, beta=0.05)) 

種となる単語を入力してください: タラ
['ジャガイモ', 'ホウレンソウ', 'ソラマメ', 'ソラマメ', 'キャベツ', 'インゲンマメ', 'ダイコン', 'ソラマメ', 'ハクサイ', 'キャベツ']
